In [1]:
# Install docker, docker-compose, python, pandas, postgres and dependencies

!pip install -q pandas fastparquet sqlalchemy psycopg2-binary requests

In [2]:
!pip freeze > requirements.txt

## GET DATA FROM API ##

> I want the database to have under 10000 entries

In [ ]:
import requests
import os
import argparse

import pandas as pd

from sqlalchemy import create_engine


In [ ]:
# Function to fetch data from Art Institute of Chicago API
def fetch_art_data():
    """
    Fetch data from Art Institute of Chicago API.
    Args:
        limit (int): Number of records to fetch.
    Returns:
        list: List of exhibition data.
    """
    response = requests.get(ART_API_URL.format(LIMIT=LIMIT))
    response.raise_for_status()

    data = response.json()
    
    return data['data']

# Fetch and process data
def process_data():
    """
    Fetch data and convert to DataFrame.
    Returns:
        pd.DataFrame: Processed DataFrame.
    """
    base_art_data = fetch_art_data()
    print(f"Fetched {len(base_art_data)} records.")

    base_df = pd.DataFrame(base_art_data)

    relevant_columns = ['id','title','short_description','web_url','image_url',
                               'gallery_title','artwork_ids','artwork_titles', 'artist_ids',
                               'source_updated_at','updated_at']
    df = base_df[relevant_columns]

    # Insert index column
    df.insert(0, 'index', range(1, len(df) + 1))
    return df

# df = process_data()

# print(df.sample(2)[['image_url','title']].values[0])
# df.head()

## Push the data into Postgres

In [ ]:
# Constants
 
LIMIT = 10

ART_API_URL = f'https://api.artic.edu/api/v1/exhibitions?limit={LIMIT}'

DATABASE_USERNAME = 'myuser'
DATABASE_PASSWORD = 'mypassword'
DATABASE_HOST = 'postgres'
DATABASE_PORT = '5432'
DATABASE_NAME = 'artdb'

TABLE_NAME = 'exhibitions'

COUNTER = 10000

In [ ]:
def main(params):
    """
    Main function to store data into PostgreSQL database.
    Gets parameters through argparse.
    """

    db_uri = f'postgresql+psycopg2://{params.user}:{params.password}@{params.host}:{params.port}/{params.db}'

    engine = create_engine(db_uri)

    df = process_data()
    df.to_sql(name=params.table_name, 
              con=engine, 
              if_exists='replace', 
              index=False)
    NUM_ENTRIES = len(df)
    print(f"{NUM_ENTRIES} Data stored in table '{params.table_name}' successfully.")

    while NUM_ENTRIES < COUNTER:
        df = process_data()
        df.to_sql(name=params.table_name, 
                  con=engine, 
                  if_exists='append', 
                  index=False)
        NUM_ENTRIES += len(df)
        print(f"{NUM_ENTRIES} Data stored in table '{params.table_name}' successfully.")


In [ ]:
# Define the arguments as variables (replace these with your actual values or widgets)
DATABASE_USERNAME = 'myuser'
DATABASE_PASSWORD = 'mypassword'
DATABASE_NAME = 'artdb'
TABLE_NAME = 'exhibitions'

# Simulate the args namespace as a simple class or use a dictionary
class Args:
    def __init__(self, user, password, host, port, db, table_name):
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.db = db
        self.table_name = table_name


In [ ]:

# Create an args instance with your parameters
args = Args(
    user=DATABASE_USERNAME,
    password=DATABASE_PASSWORD,
    host='localhost',  # or 'db' if running in Docker
    port=5432,
    db=DATABASE_NAME,
    table_name=TABLE_NAME
)


In [ ]:
main(args)


In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Store Art Institute of Chicago data into PostgreSQL database.")
    parser.add_argument('--user', type=str, default=f'{DATABASE_USERNAME}', help='Database username')
    parser.add_argument('--password', type=str, default=f'{DATABASE_PASSWORD}', help='Database password')
    parser.add_argument('--host', type=str, default='localhost', help='Database host')
    parser.add_argument('--port', type=int, default=5432, help='Database port')
    parser.add_argument('--db', type=str, default=f'{DATABASE_NAME}', help='Database name to connect to')
    parser.add_argument('--table_name', type=str, default=TABLE_NAME, help='Table name to store data')
    
    args = parser.parse_args()

    main(args)



In [ ]:
!jupyter nbconvert --to script main.ipynb --output-dir=app --output main_getter.py